In [1]:
import pickle

from src.evaluation.evaluator_manager import EvaluatorManager
from src.evaluation.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.evaluation.evaluator_manager_triplets import EvaluatorManager as TripletsEvaluatorManager
from src.utils.context import Context

from utils_martina.my_utils import *

In [2]:
import torch
torch.manual_seed(0)

import random
random.seed(0) 

import numpy as np
np.random.seed(0)

In [3]:
# Set patient and record (same channels)

observations = ["chb01_03", "chb01_04", "chb01_15", "chb01_16", "chb01_18", "chb01_21", "chb01_26"]
# observations = ["chb03_01", "chb03_02", "chb03_03", "chb03_04", "chb03_34", "chb03_35", "chb03_36"]
observations = ["chb01_03", "chb01_04"]

# observations = ["chb02_16", "chb02_19"]
# observations = ["chb05_06", "chb05_13", "chb05_16", "chb05_17", "chb05_22"]
# observations = ["chb06_01", "chb06_04"] #, "chb06_09", "chb06_10", "chb06_13", "chb06_18", "chb06_24"]
# observations = ["PN00_1", "PN00_2", "PN00_4", "PN00_5"]
# observations = ["PN14_1", "PN14_2", "PN14_3", "PN14_4"]

create_dataset_json(observations)

In [4]:
logs_path = "..\\..\\explainability\GRETEL-repo\\output\\logs\\"
eval_manager_path = "..\\..\\explainability\GRETEL-repo\\output\\eval_manager\\"

## Removing cache data

In [5]:
remove_cache = True

if remove_cache:
    list_folders = [".\data\cache\datasets", ".\data\cache\explainers", ".\data\cache\oracles"]

    for folder in list_folders:
        for file in os.listdir(folder):
            if file.startswith("EEG-"):  # Checks if the file begins with "EEG-"
                cache_path = os.path.join(folder, file)
                if os.path.isfile(cache_path):
                    os.remove(cache_path)
                    print(f"Removed file: {cache_path}")
                elif os.path.isdir(cache_path):
                    shutil.rmtree(cache_path)
                    print(f"Removed folder: {cache_path}")

## Run Gretel experiment

In [6]:
path = 'config/EEG-CORR-topk.jsonc'

context = Context.get_context(path)
context.run_number = -1

context.logger.info(f"Evaluating patients: {observations}")
context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
eval_manager = PairedEvaluatorManager(context)

eval_manager.evaluate()

2025-06-30 17:00:25,237 | INFO | 22268 - Evaluating patients: ['chb01_03', 'chb01_04']
2025-06-30 17:00:25,273 | INFO | 22268 - Executing: config/EEG-CORR-topk.jsonc Run: -1
2025-06-30 17:00:25,353 | INFO | 22268 - Loading: EEG_CORR-2f1150e5d1acd45b031014443073b786
2025-06-30 17:00:25,420 | INFO | 22268 - Created: EEG_CORR-2f1150e5d1acd45b031014443073b786
2025-06-30 17:00:25,436 | INFO | 22268 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2025-06-30 17:00:25,453 | INFO | 22268 - Instantiating: torch.optim.RMSprop
2025-06-30 17:00:25,455 | INFO | 22268 - Instantiating: torch.nn.CrossEntropyLoss
2025-06-30 17:00:25,513 | INFO | 22268 - Creating: OracleTorch-5dcfbb7664b74227753343067ffaa695
2025-06-30 17:00:26,086 | INFO | 22268 - epoch = 0 ---> loss = 0.6838	 accuracy = 0.5104
2025-06-30 17:00:26,385 | INFO | 22268 - epoch = 1 ---> loss = 0.6535	 accuracy = 0.5156
2025-06-30 17:00:26,666 | INFO | 22268 - epoch = 2 ---> loss = 0.6139	 accuracy = 0.6133
2025-06-30 17:00:26,945 | INFO | 

In [7]:
# Save eval_manager to file
file_name = get_most_recent_file(logs_path).split('.')[0]

with open(eval_manager_path + file_name + ".pkl", 'wb') as f:
    pickle.dump(eval_manager, f)

## _______________________________________________________________

In [8]:
import datetime

now = datetime.datetime.now()
print("Last full run:", now.strftime("%d/%m/%Y, ore %H:%M"))

Last full run: 30/06/2025, ore 17:15


In [9]:
import winsound

winsound.Beep(440, 500)
winsound.Beep(400, 500)
winsound.Beep(440, 500)

In [10]:
# # Indici test
# fold_id = eval_manager._evaluators[0]._explainer.fold_id
# print(eval_manager._evaluators[0].dataset.splits[fold_id]['test'])